In [32]:
# save cinema locations in Prague

import urllib2
import json
import api_key
import pandas as pd
import colorsys
import time

requestURL = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query=cinemas%20in%20Prague&key=' + api_key.API_KEY

def parse_next(json_data, names, lats, lngs):
    parsed = 0
    for result in json_data['results']:
        if (('name' in result) and ('geometry' in result) and ('location' in result['geometry'])
            and ('lat' in result['geometry']['location']) and ('lng' in result['geometry']['location'])):
            names.append(result['name'])
            lats.append(result['geometry']['location']['lat'])
            lngs.append(result['geometry']['location']['lng'])
            parsed += 1
    return parsed

names = []
lats = []
lngs = []
parsed = 0
to_parse = 200
next_page_token = None

places = pd.DataFrame()

while parsed < to_parse:
    new_address = requestURL
    if next_page_token is not None:
        new_address = requestURL + '&pagetoken=' + str(next_page_token)

    #print(new_address)
    html = urllib2.urlopen(new_address).read().decode('utf-8')
    data = json.loads(html)
    parsed += parse_next(data, names, lats, lngs)

    if 'next_page_token' not in data:
        break
    
    next_page_token = data['next_page_token']
    time.sleep(5)
    
places['name'] = names
places['lat'] = lats
places['lng'] = lngs
places.to_csv("cinema_places.csv", encoding='utf-8')

In [49]:
# parse distances by car

import urllib2
import json
import api_key
import pandas as pd
import colorsys
import time

requestURL = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
key = api_key.API_KEY

my_location = 'Pelc+Tyrolka'
dest_location = ''

def parse_next(json_data, name, lat, lng, distances_names, distances_time, distances_km, distances_lats, distances_lngs):
    if 'rows' not in data:
        return

    element = data['rows']
    if len(element) == 0:
        return

    element = element[0]
    if 'elements' not in element:
        return

    element = element['elements']
    if len(element) == 0:
        return

    element = element[0]
    if 'distance' in element:
        distances_names.append(name)
        distances_time.append(element['duration']['value'])
        distances_km.append(element['distance']['value'])
        distances_lats.append(lat)
        distances_lngs.append(lng)

        
distances_names = []
distances_time = []
distances_km = []
distances_lats = []
distances_lngs = []

places = pd.DataFrame()

i = 0
for lat,lng in zip(lats, lngs):
    address = str(lat) + ',' + str(lng)
    new_address = requestURL + 'origins=' + my_location + '&destinations=' + address.replace(' ', '+') + '&key=' + key

    # print(new_address)
    html = urllib2.urlopen(new_address).read().decode('utf-8')
    data = json.loads(html)
    parse_next(data, names[i], lats[i], lngs[i], distances_names, distances_time, distances_km, distances_lats, distances_lngs)
    
    i += 1
    time.sleep(2)
    
places['name'] = distances_names
places['distances_time'] = distances_time
places['distances_km'] = distances_km
places['lats'] = distances_lats
places['lngs'] = distances_lngs
places.to_csv("cinema_distances_car.csv", encoding='utf-8')

In [52]:
# parse distances by transit (bus, train)

import urllib2
import json
import api_key
import pandas as pd
import colorsys
import time

requestURL = 'https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&'
key = api_key.API_KEY

my_location = 'Pelc+Tyrolka'
dest_location = ''

def parse_next(json_data, name, lat, lng, distances_names, distances_time, distances_km, distances_lats, distances_lngs):
    if 'rows' not in data:
        return

    element = data['rows']
    if len(element) == 0:
        return

    element = element[0]
    if 'elements' not in element:
        return

    element = element['elements']
    if len(element) == 0:
        return

    element = element[0]
    if 'distance' in element:
        distances_names.append(name)
        distances_time.append(element['duration']['value'])
        distances_km.append(element['distance']['value'])
        distances_lats.append(lat)
        distances_lngs.append(lng)
        
distances_names = []
distances_time = []
distances_km = []
distances_lats = []
distances_lngs = []

places_transit = pd.DataFrame()

i = 0
for lat,lng in zip(lats, lngs):
    address = str(lat) + ',' + str(lng)
    new_address = requestURL + 'origins=' + my_location + '&destinations=' + address.replace(' ', '+') + '&key=' + key

    # print(new_address)
    html = urllib2.urlopen(new_address).read().decode('utf-8')
    data = json.loads(html)
    parse_next(data, names[i], lats[i], lngs[i], distances_names, distances_time, distances_km, distances_lats, distances_lngs)
    
    i += 1
    time.sleep(2)
    
places_transit['name'] = distances_names
places_transit['distances_time'] = distances_time
places_transit['distances_km'] = distances_km
places_transit['lats'] = distances_lats
places_transit['lngs'] = distances_lngs
places_transit.to_csv("cinema_distances_transit.csv", encoding='utf-8')

In [55]:
print(places)

                                     name  distances_time  distances_km  \
0                             Cinema City             815          6129   
1                               Kino Aero             778          8120   
2                                 BIO OKO             445          2891   
3                           Kino Světozor             689          5656   
4                             Cinema City             853          6563   
5         Premiere Cinemas Praha Hostivař            1369         23252   
6                            Kino Lucerna             755          6280   
7                              Kino Atlas             641          4488   
8                      Komorní kino Evald             843          7022   
9                             Cinema City             900         10960   
10                                    Mat             797          6541   
11                     CINEMA CITY Zličín            1282         17182   
12  Village Cinemas Czech

In [56]:
print(places_transit)

                                     name  distances_time  distances_km  \
0                             Cinema City            1738          4936   
1                               Kino Aero            2532          8517   
2                                 BIO OKO            1135          3200   
3                           Kino Světozor            1932          5883   
4                             Cinema City            2126          7285   
5         Premiere Cinemas Praha Hostivař            3786         14375   
6                            Kino Lucerna            1983          5930   
7                              Kino Atlas            1449          3810   
8                      Komorní kino Evald            1913          6288   
9                             Cinema City            2373          7474   
10                                    Mat            1988          7079   
11                     CINEMA CITY Zličín            3723         19504   
12  Village Cinemas Czech